프롬프트 엔지니어링
llm에서 테스트 데이터 생성 및 확인
아키텍쳐

In [24]:
import openai
import re
import os
import emoji

# ✅ OpenAI API Key 설정 (환경변수 활용 추천)
api_key = ''

client = openai.OpenAI(api_key=api_key)

def remove_emojis(text):
    """이모지 및 비ASCII 문자 제거"""
    text = emoji.replace_emoji(text, replace='')  # 이모지 제거
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # 비ASCII 문자 제거
    return text

def get_keyword(user_input):   
    """OpenAI API를 사용하여 5개의 키워드를 추출하는 함수"""
    try:
        # ✅ 시스템 프롬프트 정의
        system_prompt = f"""
        너는 사용자에게 스마트폰을 추천해주는 사람이야.

        아래 기준을 꼭 지켜줘

        [기준]
        1. {user_input}에 필요한 스마트폰 기능 또는 사양, 부품을 추천해야해
        2. 어떠한 설명이나 부가적인 말들은 제거하고 꼭 필요한 키워드 5개만 제공해줘
        3. 키워드 5개는 쉼표(,)로 구분해서 제공해줘
        4. 이 사항을 꼭 숙지해줘

        [예시]
        user_input : '영화 보기'
        
        원하는 결과 : 배터리, 휴대폰 크기, 화질, 저장용량, 스피커 품질
        """

        # 사용자 입력 포함
        user_prompt = f'휴대폰을 살때 고려해야하는 점에서 "{user_input}과 관련된 키워드 5개를 선별해줘.'

        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1  # 정답이 일관되게 나오도록 설정
        )

        # 응답 데이터 처리
        raw_output = response.choices[0].message.content.strip()
        raw_output = raw_output.replace("'",'').split(',')
        return [res.strip() for res in raw_output]

    except Exception as e:
        print(f"오류 발생: {e}")
        return 0

# def main(keyword):
#     """키워드를 5개 정확히 추출하는 메인 루프"""
#     while True:
#         res = get_keyword(keyword)

#         if len(res) == 5 and all(res):  # ✅ 5개이며 공백이 없는 경우만 성공
#             print("✅ 5개 추출 성공:", res)
#             break
#         else:
#             print("🔄 다시 시도 중...")


In [35]:
output = get_keyword('영상 제작')
output

'화질, 안정감, 음질, 속도, 저장공간'

In [30]:
output = get_keyword('운동하기')
output

['배터리', '방수 기능', 'GPS', '내장 센서', '내구성']

In [80]:
output = get_keyword('만화책 읽기')
output

['화면 크기', '해상도', '눈 건강', '배터리 수명', '저장 용량']

In [35]:
list_list = ['영상 제작', '사진 촬영', '디지털 아트', '음악 & 사운드', '쇼핑 & 패션', 'SNS & 커뮤니티', '여행 & 탐방', '모바일 게임', '독서 & 필기', '운동 & 건강']


cnt = 0
while cnt  < 3:
    for i in list_list:
        for j in range(0, 3):
            output = get_keyword(i)
            print(f'''
            ▶ {j}번째
            
            ▶ 선택한 영역
            {i}
    
            ▶ LLM이 뽑아낸 데이터
            {output}
            ''')
    cnt += 1


            ▶ 0번째
            
            ▶ 선택한 영역
            영상 제작
    
            ▶ LLM이 뽑아낸 데이터
            ['카메라 해상도', '카메라 안정화', '저장용량', 'RAM 용량', '화면 해상도']
            

            ▶ 1번째
            
            ▶ 선택한 영역
            영상 제작
    
            ▶ LLM이 뽑아낸 데이터
            ['카메라 해상도', '카메라 안정화', '화면 해상도', '저장 용량', 'RAM 용량']
            

            ▶ 2번째
            
            ▶ 선택한 영역
            영상 제작
    
            ▶ LLM이 뽑아낸 데이터
            ['카메라 해상도', '카메라 안정화', '화면 해상도', '저장 용량', 'RAM 용량']
            

            ▶ 0번째
            
            ▶ 선택한 영역
            사진 촬영
    
            ▶ LLM이 뽑아낸 데이터
            ['카메라 해상도', '렌즈 종류', '광각 카메라', '광학 줌', '촬영 모드']
            

            ▶ 1번째
            
            ▶ 선택한 영역
            사진 촬영
    
            ▶ LLM이 뽑아낸 데이터
            ['카메라 해상도', '광각 렌즈', '조리개 값', '광항성', '광항성능']
            

            ▶ 2번째
            
            ▶ 선택한 영역
            사진 촬영
    
            ▶ LLM이 뽑아낸 데이터
         

KeyboardInterrupt: 

In [83]:
import pandas as pd 
user_inputs = ['영상 제작', '사진 촬영', '디지털 아트', '음악 & 사운드', '쇼핑 & 패션', 'SNS & 커뮤니티', '여행 & 탐방', '모바일 게임', '독서 & 필기', '운동 & 건강']

res = []
cnt = 0
while cnt < 3:
    for user_input in user_inputs:
        output = get_keyword(user_input)
        print(f'''
        ▶ user_input
        {user_input}
        
        ▶ output
        {output}
        ''')
        res_output = []
        for i in output:
            res_output.append(i.split(','))
        res.append([user_input, res_output])
    cnt += 1



        ▶ user_input
        영상 제작
        
        ▶ output
        ['카메라 해상도', '화면 크기', 'RAM 용량', '저장 용량', '프로세서 성능']
        

        ▶ user_input
        사진 촬영
        
        ▶ output
        ['카메라 해상도', '렌즈 종류', '광각 카메라', '광학 줌', '촬영 모드']
        

        ▶ user_input
        디지털 아트
        
        ▶ output
        ['화면 해상도', '그래픽 성능', 'RAM 용량', '저장 용량', '전면 카메라 해상도']
        

        ▶ user_input
        음악 & 사운드
        
        ▶ output
        ['음질', '스피커 품질', '이어폰 단자', 'DAC', '오디오 코덱']
        

        ▶ user_input
        쇼핑 & 패션
        
        ▶ output
        ['카메라 품질', '화면 크기', '디자인', '색상 다양성', '저장 용량']
        

        ▶ user_input
        SNS & 커뮤니티
        
        ▶ output
        ['카메라 품질', '배터리 수명', '화면 크기', '저장 용량', 'RAM 용량']
        

        ▶ user_input
        여행 & 탐방
        
        ▶ output
        ['배터리 수명', '카메라 화질', '내장 저장공간', '방수 기능', 'GPS 정확도']
        

        ▶ user_input
        모바일 게임
        
        ▶ output
        ['RAM', '그래픽 카드', '

In [85]:
pd.DataFrame(res, columns = ['user_input', 'outputs']).sort_values('user_input', ascending = False).to_csv('res_LLM_3.5.csv', encoding = 'utf-8', index = False)

In [88]:
import openai
import re
import os
import emoji

# ✅ OpenAI API Key 설정 (환경변수 활용 추천)
api_key = ''

client = openai.OpenAI(api_key=api_key)

def remove_emojis(text):
    """이모지 및 비ASCII 문자 제거"""
    text = emoji.replace_emoji(text, replace='')  # 이모지 제거
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # 비ASCII 문자 제거
    return text

def get_keyword(user_input):   
    """OpenAI API를 사용하여 5개의 키워드를 추출하는 함수"""
    try:
        # ✅ 시스템 프롬프트 정의
        system_prompt = f"""
        너는 사용자에게 스마트폰을 추천해주는 사람이야.

        아래 기준을 꼭 지켜줘

        [기준]
        1. {user_input}에 필요한 스마트폰 기능 또는 사양, 부품을 추천해야해
        2. 어떠한 설명이나 부가적인 말들은 제거하고 꼭 필요한 키워드 5개만 제공해줘
        3. 키워드 5개는 쉼표(,)로 구분해서 제공해줘
        4. 이 사항을 꼭 숙지해줘

        [예시]
        user_input : '영화 보기'
        
        원하는 결과 : 배터리, 휴대폰 크기, 화질, 저장용량, 스피커 품질
        """

        # 사용자 입력 포함
        user_prompt = f'휴대폰을 살때 고려해야하는 점에서 "{user_input}과 관련된 키워드 5개를 선별해줘.'

        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1  # 정답이 일관되게 나오도록 설정
        )

        # 응답 데이터 처리
        raw_output = response.choices[0].message.content.strip()
        raw_output = raw_output.replace("'",'').split(',')
        return [res.strip() for res in raw_output]

    except Exception as e:
        print(f"오류 발생: {e}")
        return 0

# def main(keyword):
#     """키워드를 5개 정확히 추출하는 메인 루프"""
#     while True:
#         res = get_keyword(keyword)

#         if len(res) == 5 and all(res):  # ✅ 5개이며 공백이 없는 경우만 성공
#             print("✅ 5개 추출 성공:", res)
#             break
#         else:
#             print("🔄 다시 시도 중...")
import pandas as pd 
user_inputs = ['영상 제작', '사진 촬영', '디지털 아트', '음악 & 사운드', '쇼핑 & 패션', 'SNS & 커뮤니티', '여행 & 탐방', '모바일 게임', '독서 & 필기', '운동 & 건강']

res = []
cnt = 0
while cnt < 3:
    for user_input in user_inputs:
        output = get_keyword(user_input)
        print(f'''
        ▶ user_input
        {user_input}
        
        ▶ output
        {output}
        ''')
        res_output = []
        for i in output:
            res_output.append(i.split(','))
        res.append([user_input, res_output])
    cnt += 1

pd.DataFrame(res, columns = ['user_input', 'outputs']).sort_values('user_input', ascending = False).to_csv('res_LLM_4o.csv', encoding = 'utf-8', index = False)



        ▶ user_input
        영상 제작
        
        ▶ output
        ['카메라 화질', '저장용량', '프로세서 성능', '배터리 수명', '디스플레이 품질']
        

        ▶ user_input
        사진 촬영
        
        ▶ output
        ['카메라 화소', '조리개 값', '이미지 센서', '광학 줌', '손떨림 보정']
        

        ▶ user_input
        디지털 아트
        
        ▶ output
        ['디스플레이 품질', '스타일러스 지원', '프로세서 성능', '저장용량', '배터리 수명']
        

        ▶ user_input
        음악 & 사운드
        
        ▶ output
        ['스피커 품질', '오디오 칩셋', '이어폰 잭', '블루투스 버전', '음질 설정']
        

        ▶ user_input
        쇼핑 & 패션
        
        ▶ output
        ['카메라 품질', '디스플레이', '저장용량', '앱 호환성', '배터리 지속시간']
        

        ▶ user_input
        SNS & 커뮤니티
        
        ▶ output
        ['카메라 품질', '배터리 수명', '저장용량', '화면 크기', '앱 호환성']
        

        ▶ user_input
        여행 & 탐방
        
        ▶ output
        ['배터리 수명', '카메라 품질', '내비게이션 기능', '방수 기능', '저장 용량']
        

        ▶ user_input
        모바일 게임
        
        ▶ output
        ['프로세서', 'RA

In [6]:
import openai
import re
import os
import emoji

# ✅ OpenAI API Key 설정 (환경변수 활용 추천)
api_key = ''

client = openai.OpenAI(api_key=api_key)

def remove_emojis(text):
    """이모지 및 비ASCII 문자 제거"""
    text = emoji.replace_emoji(text, replace='')  # 이모지 제거
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # 비ASCII 문자 제거
    return text

def get_keyword(user_input):   
    """OpenAI API를 사용하여 5개의 키워드를 추출하는 함수"""
    try:
        # ✅ 시스템 프롬프트 정의
        system_prompt = f"""
        너는 사용자에게 스마트폰을 추천해주는 사람이야.

        아래 기준을 꼭 지켜줘

        [기준]
        1. {user_input}에 필요한 스마트폰 기능 또는 사양, 부품을 추천해야해
        2. 어떠한 설명이나 부가적인 말들은 제거하고 꼭 필요한 키워드 5개만 제공해줘
        3. 키워드 5개는 쉼표(,)로 구분해서 제공해줘
        4. 키워드 5개는 기존에 너가 답변한 것 외에서 다양하게 생성해줘
        5. 이 사항을 꼭 숙지해줘

        [예시]
        user_input : '영화 보기'
        
        원하는 결과 : 배터리, 휴대폰 크기, 화질, 저장용량, 스피커 품질
        """

        # 사용자 입력 포함
        user_prompt = f'휴대폰을 살때 고려해야하는 점에서 "{user_input}과 관련된 키워드 5개를 선별해줘.'

        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1  # 정답이 일관되게 나오도록 설정
        )

        # 응답 데이터 처리
        raw_output = response.choices[0].message.content.strip()
        raw_output = raw_output.replace("'",'').split(',')
        return [res.strip() for res in raw_output]

    except Exception as e:
        print(f"오류 발생: {e}")
        return 0

# def main(keyword):
#     """키워드를 5개 정확히 추출하는 메인 루프"""
#     while True:
#         res = get_keyword(keyword)

#         if len(res) == 5 and all(res):  # ✅ 5개이며 공백이 없는 경우만 성공
#             print("✅ 5개 추출 성공:", res)
#             break
#         else:
#             print("🔄 다시 시도 중...")
import pandas as pd 
user_inputs = ['영상 제작', '사진 촬영', '디지털 아트', '음악 & 사운드', '쇼핑 & 패션', 'SNS & 커뮤니티', '여행 & 탐방', '모바일 게임', '독서 & 필기', '운동 & 건강']

res = []
cnt = 0
while cnt < 3:
    for user_input in user_inputs:
        output = get_keyword(user_input)
        print(f'''
        ▶ user_input
        {user_input}
        
        ▶ output
        {output}
        ''')
        res_output = []
        print(f'output을 출력해보자 {output}')
        for i in output:
            print(f'i를 출력해보자 {i}')
            res_output.append(i.split(','))
        res.append([user_input, res_output])
    cnt += 1

pd.DataFrame(res, columns = ['user_input', 'outputs']).sort_values('user_input', ascending = False).to_csv('res_LLM_3.5_prompt_ver2.csv', encoding = 'utf-8', index = False)



        ▶ user_input
        영상 제작
        
        ▶ output
        ['카메라 해상도', '촬영 안정성', '렌즈 다양성', '화면 해상도', '저장 용량']
        
output을 출력해보자 ['카메라 해상도', '촬영 안정성', '렌즈 다양성', '화면 해상도', '저장 용량']
i를 출력해보자 카메라 해상도
i를 출력해보자 촬영 안정성
i를 출력해보자 렌즈 다양성
i를 출력해보자 화면 해상도
i를 출력해보자 저장 용량

        ▶ user_input
        사진 촬영
        
        ▶ output
        ['카메라 해상도', '조리개 값', '광학 줌', '셀카 카메라 해상도', '광학 이미지 안정화']
        
output을 출력해보자 ['카메라 해상도', '조리개 값', '광학 줌', '셀카 카메라 해상도', '광학 이미지 안정화']
i를 출력해보자 카메라 해상도
i를 출력해보자 조리개 값
i를 출력해보자 광학 줌
i를 출력해보자 셀카 카메라 해상도
i를 출력해보자 광학 이미지 안정화

        ▶ user_input
        디지털 아트
        
        ▶ output
        ['화면 해상도', '그래픽 성능', 'RAM 용량', '저장 공간', '화면 크기']
        
output을 출력해보자 ['화면 해상도', '그래픽 성능', 'RAM 용량', '저장 공간', '화면 크기']
i를 출력해보자 화면 해상도
i를 출력해보자 그래픽 성능
i를 출력해보자 RAM 용량
i를 출력해보자 저장 공간
i를 출력해보자 화면 크기

        ▶ user_input
        음악 & 사운드
        
        ▶ output
        ['음악 품질', '이어폰 단자', '스테레오 스피커', 'DAC 칩', '블루투스 코덱']
        
output을 출력해보자 ['음악 품질', '이어폰

In [8]:
import openai
import re
import os
import emoji

# ✅ OpenAI API Key 설정 (환경변수 활용 추천)
api_key = ''

client = openai.OpenAI(api_key=api_key)

def remove_emojis(text):
    """이모지 및 비ASCII 문자 제거"""
    text = emoji.replace_emoji(text, replace='')  # 이모지 제거
    text = re.sub(r'[^\x00-\x7F]+', '', text)  # 비ASCII 문자 제거
    return text

def get_keyword(user_input):   
    """OpenAI API를 사용하여 5개의 키워드를 추출하는 함수"""
    try:
        # ✅ 시스템 프롬프트 정의
        system_prompt = f"""
        너는 사용자에게 스마트폰을 추천해주는 사람이야.

        아래 기준을 꼭 지켜줘

        [기준]
        1. {user_input}에 필요한 스마트폰 기능 또는 사양, 부품을 추천해야해
        2. 어떠한 설명이나 부가적인 말들은 제거하고 꼭 필요한 키워드 5개만 제공해줘
        3. 키워드 5개는 쉼표(,)로 구분해서 제공해줘
        4. 키워드 5개는 기존에 너가 답변한 것 외에서 다양하게 생성해줘
        5. 이 사항을 꼭 숙지해줘

        [예시]
        user_input : '영화 보기'
        
        원하는 결과 : 배터리, 휴대폰 크기, 화질, 저장용량, 스피커 품질
        """

        # 사용자 입력 포함
        user_prompt = f'휴대폰을 살때 고려해야하는 점에서 "{user_input}과 관련된 키워드 5개를 선별해줘.'

        # OpenAI API 호출
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0.1  # 정답이 일관되게 나오도록 설정
        )

        # 응답 데이터 처리
        raw_output = response.choices[0].message.content.strip()
        raw_output = raw_output.replace("'",'').split(',')
        return [res.strip() for res in raw_output]

    except Exception as e:
        print(f"오류 발생: {e}")
        return 0

# def main(keyword):
#     """키워드를 5개 정확히 추출하는 메인 루프"""
#     while True:
#         res = get_keyword(keyword)

#         if len(res) == 5 and all(res):  # ✅ 5개이며 공백이 없는 경우만 성공
#             print("✅ 5개 추출 성공:", res)
#             break
#         else:
#             print("🔄 다시 시도 중...")
import pandas as pd 
user_inputs = ['영상 제작', '사진 촬영', '디지털 아트', '음악 & 사운드', '쇼핑 & 패션', 'SNS & 커뮤니티', '여행 & 탐방', '모바일 게임', '독서 & 필기', '운동 & 건강']

res = []
cnt = 0
while cnt < 3:
    for user_input in user_inputs:
        output = get_keyword(user_input)
        print(f'''
        ▶ user_input
        {user_input}
        
        ▶ output
        {output}
        ''')
        res_output = []
        print(f'output을 출력해보자 {output}')
        for i in output:
            print(f'i를 출력해보자 {i}')
            res_output.append(i.split(','))
        res.append([user_input, res_output])
    cnt += 1

pd.DataFrame(res, columns = ['user_input', 'outputs']).sort_values('user_input', ascending = False).to_csv('res_LLM_4o_prompt_ver2.csv', encoding = 'utf-8', index = False)



        ▶ user_input
        영상 제작
        
        ▶ output
        ['카메라 화질', '동영상 안정화', '저장용량', '프로세서 성능', '배터리 용량']
        
output을 출력해보자 ['카메라 화질', '동영상 안정화', '저장용량', '프로세서 성능', '배터리 용량']
i를 출력해보자 카메라 화질
i를 출력해보자 동영상 안정화
i를 출력해보자 저장용량
i를 출력해보자 프로세서 성능
i를 출력해보자 배터리 용량

        ▶ user_input
        사진 촬영
        
        ▶ output
        ['카메라 화소', '조리개 값', '이미지 안정화', '줌 기능', '야간 모드']
        
output을 출력해보자 ['카메라 화소', '조리개 값', '이미지 안정화', '줌 기능', '야간 모드']
i를 출력해보자 카메라 화소
i를 출력해보자 조리개 값
i를 출력해보자 이미지 안정화
i를 출력해보자 줌 기능
i를 출력해보자 야간 모드

        ▶ user_input
        디지털 아트
        
        ▶ output
        ['디스플레이 해상도', '스타일러스 지원', '프로세서 성능', '색 재현율', '저장 용량']
        
output을 출력해보자 ['디스플레이 해상도', '스타일러스 지원', '프로세서 성능', '색 재현율', '저장 용량']
i를 출력해보자 디스플레이 해상도
i를 출력해보자 스타일러스 지원
i를 출력해보자 프로세서 성능
i를 출력해보자 색 재현율
i를 출력해보자 저장 용량

        ▶ user_input
        음악 & 사운드
        
        ▶ output
        ['스피커 품질', '오디오 칩셋', '헤드폰 잭', '블루투스 버전', '음질 설정']
        
output을 출력해보자 ['스피커 품질', '오디오 칩셋', '헤드폰

In [1]:
import os
from pyspark.sql import SparkSession

host_ip = '15.168.221.131'
DATABASE = 'SNS_DB'
user_id = 'lab13'
user_password = 'lab13'

# 환경변수 설정
os.environ['PYSPARK_SUBMIT_ARGS'] = '--jars /usr/local/lib/mysql-connector-java-5.1.49-bin.jar pyspark-shell'

# python_path = "/home/ubuntu/anaconda3/envs/Project/bin/python"
# os.environ["PYSPARK_PYTHON"] = python_path
# os.environ["PYSPARK_DRIVER_PYTHON"] = python_path

ss = SparkSession\
            .builder\
            .appName('MySQL')\
            .config('spark.executor.memory', '6g')\
            .config('spark.driver.memory', '6g')\
            .getOrCreate()
ss
mysql_url = f'jdbc:mysql://{host_ip}:3306/{DATABASE}?useSSL=false&allowPublicKeyRetrieval=true'

2025-02-13 14:23:16,263 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
df = ss.read.format('jdbc')\
                .options(
                    url = mysql_url
                    , driver = 'com.mysql.jdbc.Driver'
                    , dbtable = 'tbCrawled_Danawa'
                    , user = user_id
                    , password = user_password
                        )\
            .load()

In [4]:
df

DataFrame[id: int, scoring: string, market: string, purchasing_date: date, review_title: string, review_content: string, item: string, inserted_at: timestamp]

In [8]:
from pyspark.sql.functions import col

df = df.withColumn("purchasing_date", col("purchasing_date").cast("timestamp"))
df = df.withColumn("inserted_at", col("inserted_at").cast("timestamp"))

df

DataFrame[id: int, scoring: string, market: string, purchasing_date: timestamp, review_title: string, review_content: string, item: string, inserted_at: timestamp]

In [9]:
df.show()

+---+-------+--------+-------------------+---------------------------------+----------------------------------+------------------------------+-------------------+
| id|scoring|  market|    purchasing_date|                     review_title|                    review_content|                          item|        inserted_at|
+---+-------+--------+-------------------+---------------------------------+----------------------------------+------------------------------+-------------------+
|  1|   60점|삼성닷컴|2024-07-26 00:00:00|   
S 펜 케이스를 선택하게 된 ...|   S 펜 케이스를 선택하게 된 이...|삼성전자 갤럭시Z 폴드6 S펜 ...|2025-02-08 17:45:59|
|  2|  100점|삼성닷컴|2024-08-05 00:00:00|     
1. S펜 수납 기능가장 큰 ...|      1. S펜 수납 기능
가장 큰 ...|삼성전자 갤럭시Z 폴드6 S펜 ...|2025-02-08 17:45:59|
|  3|   80점|삼성닷컴|2024-08-26 00:00:00|
투명하고 가볍고 저렴한 케이스...|투명하고 가볍고 저렴한 케이스를...|삼성전자 갤럭시Z 폴드6 S펜 ...|2025-02-08 17:45:59|
|  4|   80점|삼성닷컴|2024-08-02 00:00:00|    
노트20 기종만 4년 여 사용...|    노트20 기종만 4년 여 사용한...|삼성전자 갤럭시Z 폴드6 S펜 ...|2025-02-08 17:45:59|
| 

In [5]:
#!pip install sentence-transformers
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import FloatType
import pymysql
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
df.select(['id', 'review_content'])

/home/ubuntu/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


DataFrame[id: int, review_content: string]

In [6]:
# ✅ Sentence Transformer 모델 로드
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

In [12]:
df.show()

2025-02-13 14:22:55,809 ERROR executor.Executor: Exception in task 0.0 in stage 3.0 (TID 3)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 473, in main
    raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.8 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:84)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:67)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:470)
	at org.apache.spark.InterruptibleIterator

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 473, in main
    raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.8 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.


In [10]:
# ✅ 댓글을 벡터로 변환하는 UDF 함수 정의
def get_embedding(text):
    return model.encode(text).tolist()  # PySpark는 리스트로 변환 필요

# ✅ 댓글 데이터에 벡터 추가
# df = df.withColumn("review_content_embedding", embedding_udf(col("review_content")))
# pd_df = df.toPandas()
# pd_df

# # ✅ 유사도 계산 함수 정의 (코사인 유사도)
# def calculate_similarity(comment_vec):
#     comment_vec = np.array(comment_vec).reshape(1, -1)
#     score = cosine_similarity([keyword_embedding], comment_vec)[0][0]
#     return float(score)  # PySpark에서는 float 변환 필요

# similarity_udf = udf(calculate_similarity, FloatType())

# # ✅ 유사도 컬럼 추가
# df = df.withColumn("similarity_score", similarity_udf(col("review_content")))

# df.show()
# # ✅ 유사도가 높은 순서대로 정렬
# df_sorted = df.orderBy(col("similarity_score").desc())

# df_sorted.show()


2025-02-13 14:22:47,917 ERROR executor.Executor: Exception in task 0.0 in stage 2.0 (TID 2)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 473, in main
    raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.8 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:517)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:84)
	at org.apache.spark.sql.execution.python.PythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:67)
	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.hasNext(PythonRunner.scala:470)
	at org.apache.spark.InterruptibleIterator

PythonException: 
  An exception was thrown from the Python worker. Please see the stack trace below.
Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 473, in main
    raise Exception(("Python in worker has different version %s than that in " +
Exception: Python in worker has different version 3.8 than that in driver 3.9, PySpark cannot run with different minor versions. Please check environment variables PYSPARK_PYTHON and PYSPARK_DRIVER_PYTHON are correctly set.


In [22]:
!which python

/home/ubuntu/anaconda3/envs/Project/bin/python


In [6]:
from pyspark.sql.functions import udf
from pyspark.sql.types import FloatType
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
import os
# ✅ UDF 실행 환경 Python 버전 지정
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python"

# ✅ Sentence Transformer 모델 로드
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

# ✅ 코사인 유사도 계산 함수 정의 (PySpark UDF 적용)
def calculate_similarity(comment_vec):
    comment_vec = np.array(comment_vec).reshape(1, -1)
    score = cosine_similarity([keyword_embedding], comment_vec)[0][0]
    return float(score)

similarity_udf = udf(calculate_similarity, FloatType())

# ✅ PySpark에서 UDF 실행 시 Python 버전 일치 확인
print("✅ PySpark Python 버전:", os.environ["PYSPARK_PYTHON"])


✅ PySpark Python 버전: /usr/bin/python


In [25]:
ss.stop()

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import NMF

documents = [
    "Deep learning is transforming AI research",
    "Machine learning models analyze big data",
    "Natural language processing enables AI assistants",
    "Big data analysis improves business decisions",
    "Reinforcement learning is a subset of machine learning"
]

# ✅ TF-IDF 벡터화
vectorizer = TfidfVectorizer(stop_words="english")
tfidf_matrix = vectorizer.fit_transform(documents)

# ✅ NMF 토픽 모델 생성
nmf_model = NMF(n_components=3, random_state=42)
nmf_topics = nmf_model.fit_transform(tfidf_matrix)

# ✅ 토픽별 키워드 출력
words = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(nmf_model.components_):
    top_words = [words[i] for i in topic.argsort()[:-6:-1]]
    print(f"토픽 {topic_idx+1}: {', '.join(top_words)}")


토픽 1: learning, machine, subset, reinforcement, transforming
토픽 2: ai, language, enables, natural, processing
토픽 3: big, data, improves, decisions, business


In [23]:
from bertopic import BERTopic

documents = [
    "Deep learning is transforming AI research",
    "Machine learning models analyze big data",
    "Natural language processing enables AI assistants",
    "Big data analysis improves business decisions",
    "Reinforcement learning is a subset of machine learning"
]

topic_model = BERTopic()
topics, probs = topic_model.fit_transform(documents)

print(topic_model.get_topic_info())  # 토픽 정보 출력


/home/ubuntu/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/umap/spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(
/home/ubuntu/anaconda3/envs/pytorch_p39/lib/python3.9/site-packages/umap/spectral.py:519: RuntimeWarning: k >= N for N * N square matrix. Attempting to use scipy.linalg.eigh instead.
  eigenvalues, eigenvectors = scipy.sparse.linalg.eigsh(


TypeError: Cannot use scipy.linalg.eigh for sparse A with k >= N. Use scipy.linalg.eigh(A.toarray()) or reduce k.